In [1]:
# 필요한 lib 목록들
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import os
from time import sleep
import csv
import datetime
import time
import pytz

In [7]:
# start_date = datetime(2024, 1, 1)
# end_date = datetime(2024, 1, 2)

# # datetime 객체를 타임스탬프로 변환
# start_timestamp = int(start_date.timestamp())
# end_timestamp = int(end_date.timestamp())

# Ticker = "TON"
# offset = 0
# limit = 256
# page_number = 1

# headers = {
#     "X-API-Key" : "35a58afb114a6469a9641642898eb95e519e4eb5426294b53ff6623173f74a1d"
# }

# # JSON 파일 저장 디렉토리 설정
# output_dir = f'{Ticker}_{str(start_date)[:10]}_{str(end_date)[:10]}_json_pages'
# os.makedirs(output_dir, exist_ok=True)

In [17]:
# while True:
#     url = f"https://toncenter.com/api/v3/transactions?start_utime={start_timestamp}&end_utime={end_timestamp}&limit={limit}&offset={offset}&sort=asc&api_key=35a58afb114a6469a9641642898eb95e519e4eb5426294b53ff6623173f74a1d"
#     response = requests.get(url, headers=headers)
#     if response.status_code == 200: # 문제 없을 경우 데이터가 추출
#         data = response.json()
#         transfers = data['transactions']
#         if not transfers:
#             break
#             # JSON 파일로 저장
        
#         with open(os.path.join(output_dir, f'{Ticker}_{str(start_date)[:10]}_{str(end_date)[:10]}_page_{page_number}.json'), 'w') as json_file:
#             json.dump(transfers, json_file, indent=4)
        
#         offset += limit  # 다음 페이지로 이동
#         print(f"page {page_number} is complete!")
#         page_number += 1

#     else:
#         print(f'Query failed with status code {response.status_code}')
#         print(response.text)
#         break
#     sleep(1)

# print(f'Data successfully saved to JSON files in {output_dir}')

# # JSON 파일 저장 디렉토리 설정
# input_dir = f'{Ticker}_{str(start_date)[:10]}_{str(end_date)[:10]}_json_pages'
# output_csv = f'{Ticker}_{str(start_date)[:10]}_{str(end_date)[:10]}_TRX.csv'

# # CSV 파일에 저장할 필드 이름 설정
# fieldnames = [
#     'timestamp', 'block_height', 'master_block_height', 'sender_address', 
#     'receiver_address', 'amount', 'trx_hash', 'trx_gasPrice'
# ]

# # 모든 JSON 데이터 리스트에 저장
# all_transfers = []

# # JSON 파일들 읽기
# for filename in os.listdir(input_dir):
#     if filename.endswith('.json'):
#         with open(os.path.join(input_dir, filename), 'r') as jsonfile:
#             transfers = json.load(jsonfile)

#             # JSON 데이터에서 필요한 필드 추출하여 리스트에 추가
#             for transfer in transfers:
#                 row = {           
#                     'timestamp': transfer['now'],
#                     'block_height': transfer['block_ref']['seqno'],
#                     'master_block_height' : transfer['mc_block_seqno'],
#                     'sender_address': transfer['out_msgs'][0]['source'],
#                     'receiver_address': transfer['out_msgs'][0]['destination'],
#                     'amount': transfer['out_msgs'][0]['value'],
#                     'trx_hash': transfer['hash'],
#                     'trx_gasPrice': transfer['total_fees']
#                 }
#                 all_transfers.append(row)

# # 타임스탬프를 기준으로 정렬
# all_transfers_sorted = sorted(all_transfers, key=lambda x: datetime.strptime(x['timestamp'], '%Y-%m-%d %H:%M:%S'))

# # CSV 파일 생성 및 필드 이름 작성
# with open(output_csv, 'w', newline='') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
    
#     # 정렬된 데이터 CSV 파일에 작성
#     for row in all_transfers_sorted:
#         writer.writerow(row)

# print(f'Data successfully saved to CSV file {output_csv}')
# print('----------------------------------------------------')

In [16]:
start_date = datetime.datetime(2024, 1, 1)
start_date.timestamp()

1704034800.0

In [3]:
# API 키와 기본 URL 설정
api_key = "35a58afb114a6469a9641642898eb95e519e4eb5426294b53ff6623173f74a1d"
base_url = "https://toncenter.com/api/v3/transactions"
headers = {
    "X-API-Key": api_key
}

# 시작 및 종료 날짜 설정
start_date = datetime.datetime(2024, 1, 1, tzinfo=pytz.timezone('UTC'))
end_date = datetime.datetime(2024, 1, 2, tzinfo=pytz.timezone('UTC'))

# CSV 파일에 저장할 필드 이름 설정
fieldnames = [
    'timestamp', 'block_height', 'master_block_height', 'sender_address', 
    'receiver_address', 'amount', 'trx_hash', 'trx_gasPrice'
]


# 하루 단위로 데이터를 가져오기 위해 datetime.timedelta 사용
one_day = datetime.timedelta(days=1)
current_date = start_date

while current_date < end_date:
    next_date = current_date + one_day
    
    start_timestamp = int(current_date.timestamp())
    end_timestamp = int(next_date.timestamp())
    
    # CSV 파일 경로 설정
    output_csv = f'TONCOIN_{current_date.strftime("%Y-%m-%d")}.csv'
    
    limit = 256
    offset = 0
    page_number = 1
    
    # CSV 파일 생성 및 필드 이름 작성
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        while True:
            url = f"{base_url}?start_utime={start_timestamp}&end_utime={end_timestamp}&limit={limit}&offset={offset}&sort=asc"
            try:
                response = requests.get(url, headers=headers, timeout=10)  # 타임아웃 설정 (10초)
                response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                time.sleep(5)  # 잠시 대기 후 재시도
                continue
            
            data = response.json()
            transfers = data['transactions']
            if not transfers:
                break
            
            # JSON 데이터에서 필요한 필드 추출하여 리스트에 추가
            for transfer in transfers:
                for out_msg in transfer['out_msgs']:
                    row = {
                        'timestamp': datetime.datetime.utcfromtimestamp(transfer['now']).strftime('%Y-%m-%d %H:%M:%S'),
                        'block_height': transfer['block_ref']['seqno'],
                        'master_block_height': transfer['mc_block_seqno'],
                        'sender_address': out_msg['source'],
                        'receiver_address': out_msg['destination'],
                        'amount': out_msg['value'],
                        'trx_hash': transfer['hash'],
                        'trx_gasPrice': transfer['total_fees']
                    }
                    writer.writerow(row)
            
            offset += limit  # 다음 페이지로 이동
            if page_number%50 == 0:
                print(f"Page {page_number} for date {current_date.strftime('%Y-%m-%d')} is complete!")
            page_number += 1

            # 너무 많은 요청을 하지 않기 위해 잠시 대기
            time.sleep(2)
    
    # 현재 날짜에 대한 CSV 파일 저장 완료 메시지 출력
    print(f"Data for {current_date.strftime('%Y-%m-%d')} successfully saved to {output_csv}")
    
    # 다음 날짜로 이동
    current_date = next_date

print(f'Data successfully saved to CSV files in {output_dir}')


/var/folders/9c/5r2yc1zn75l79yd3m6vybk4r0000gn/T/ipykernel_68958/4177918219.py:60: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'timestamp': datetime.datetime.utcfromtimestamp(transfer['now']).strftime('%Y-%m-%d %H:%M:%S'),


Page 50 for date 2024-01-01 is complete!
Page 100 for date 2024-01-01 is complete!
Page 150 for date 2024-01-01 is complete!
Page 200 for date 2024-01-01 is complete!
Page 250 for date 2024-01-01 is complete!
Page 300 for date 2024-01-01 is complete!
Page 350 for date 2024-01-01 is complete!
Page 400 for date 2024-01-01 is complete!
Page 450 for date 2024-01-01 is complete!
Page 500 for date 2024-01-01 is complete!
Page 550 for date 2024-01-01 is complete!
Page 600 for date 2024-01-01 is complete!
Page 650 for date 2024-01-01 is complete!
Page 700 for date 2024-01-01 is complete!
Page 750 for date 2024-01-01 is complete!
Request failed: HTTPSConnectionPool(host='toncenter.com', port=443): Max retries exceeded with url: /api/v3/transactions?start_utime=1704067200&end_utime=1704153600&limit=256&offset=197120&sort=asc (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1361c6420>: Failed to establish a new connection: [Errno 51] Network is unreachable'))
Request

NameError: name 'output_dir' is not defined